# FC - Neural Network with PCA preprocessing

In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("../data", one_hot=True) # y labels are oh-encoded

n_train = mnist.train.num_examples # 55,000
n_validation = mnist.validation.num_examples # 5000
n_test = mnist.test.num_examples # 10,000

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ../data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ../data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting ../data/t10k-images-idx3-ubyte.gz
Extracting ../data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [8]:
mnist.train.images.shape

(55000, 784)

In [3]:
from mnist import MNIST

mndata = MNIST('../data/')

train_x, train_y = mndata.load_training()
test_x, test_y = mndata.load_testing()

In [17]:
n_input = 784   # input layer (28x28 pixels)
n_hidden1 = 1024 # 1st hidden layer
n_hidden2 = 512 # 2nd hidden layer
n_hidden3 = 256 # 3rd hidden layer
n_hidden4 = 128 # 3rd hidden layer
n_output = 10   # output layer (0-9 digits)

learning_rate = 1e-4
n_iterations = 4000
batch_size = 128
dropout = 0.5
regularization = 1e-3

X = tf.placeholder("float", [None, n_input])
Y = tf.placeholder("float", [None, n_output])

covariance = tf.matmul(tf.transpose(X), X)
eigenvals, eigenvecs = tf.linalg.eigh(covariance)

_eigenvecs = tf.placeholder("float", [None, 784])

coefs = tf.matmul(X, tf.transpose(_eigenvecs))

_reX = tf.matmul(coefs, _eigenvecs)

keep_prob = tf.placeholder(tf.float32) 

weights = {
    'w1': tf.Variable(tf.truncated_normal([n_input, n_hidden1], stddev=0.1)),
    'w2': tf.Variable(tf.truncated_normal([n_hidden1, n_hidden2], stddev=0.1)),
    'w3': tf.Variable(tf.truncated_normal([n_hidden2, n_hidden3], stddev=0.1)),
    'w4': tf.Variable(tf.truncated_normal([n_hidden3, n_hidden4], stddev=0.1)),
    'out': tf.Variable(tf.truncated_normal([n_hidden4, n_output], stddev=0.1)),
}

biases = {
    'b1': tf.Variable(tf.constant(0.1, shape=[n_hidden1])),
    'b2': tf.Variable(tf.constant(0.1, shape=[n_hidden2])),
    'b3': tf.Variable(tf.constant(0.1, shape=[n_hidden3])),
    'b4': tf.Variable(tf.constant(0.1, shape=[n_hidden4])),
    'out': tf.Variable(tf.constant(0.1, shape=[n_output]))
}

layer_1 = tf.nn.relu(tf.add(tf.matmul(_reX, weights['w1']), biases['b1']))
layer_2 = tf.nn.relu(tf.add(tf.matmul(layer_1, weights['w2']), biases['b2']))
layer_3 = tf.nn.relu(tf.add(tf.matmul(layer_2, weights['w3']), biases['b3']))
layer_4 = tf.nn.relu(tf.add(tf.matmul(layer_3, weights['w4']), biases['b4']))
layer_drop   = tf.nn.dropout(layer_4, keep_prob)
output_layer = tf.matmul(layer_drop, weights['out']) + biases['out']

cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=Y, logits=output_layer))

l2norm  = tf.nn.l2_loss( weights['w1'] )
l2norm += tf.nn.l2_loss( weights['w2'] )
l2norm += tf.nn.l2_loss( weights['w3'] )
l2norm += tf.nn.l2_loss( weights['w4'] )
l2norm += tf.nn.l2_loss( weights['out'])

loss = cross_entropy + 0.5 * l2norm * regularization

# train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
train_step = tf.train.AdamOptimizer(1e-4).minimize(loss)

correct_pred = tf.equal(tf.argmax(output_layer, 1), tf.argmax(Y, 1))
pred = tf.argmax(output_layer, 1)
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

with tf.Session() as sess:
    init = tf.global_variables_initializer()
    sess.run(init)
    
    _, myeigenvecs = sess.run([eigenvals, eigenvecs], feed_dict={X: mnist.train.images})
    print(myeigenvecs.shape)
 
    # train on mini batches
    for i in range(n_iterations):
        batch_x, batch_y = mnist.train.next_batch(batch_size)
        
        sess.run(train_step, feed_dict={X: batch_x, Y: batch_y, _eigenvecs: myeigenvecs, keep_prob:dropout})
    
        # print loss and accuracy (per minibatch)
        if i%100==0:
            minibatch_loss, minibatch_accuracy = sess.run([cross_entropy, accuracy], feed_dict={X: batch_x, Y: batch_y, keep_prob:1.0})
            print("Iteration", str(i), "\t| Loss =", str(minibatch_loss), "\t| Accuracy =", str(minibatch_accuracy))
            
    test_accuracy = sess.run(accuracy, feed_dict={X: mnist.test.images, Y: mnist.test.labels, keep_prob:1.0})
    
    final_weights = {
        'w1': sess.run( weights['w1'] ),
        'w2': sess.run( weights['w2'] ),
        'w3': sess.run( weights['w3'] ),
        'w4': sess.run( weights['w4'] ),
        'out': sess.run( weights['out'] )
    }
    
    correct = sess.run(correct_pred, feed_dict={X: mnist.test.images, Y: mnist.test.labels, keep_prob:1.0})
    
    _pred = sess.run(pred, feed_dict={X: mnist.test.images, Y: mnist.test.labels, keep_prob:1.0})
    
    print("\nAccuracy on test set:", test_accuracy)

(784, 784)


InvalidArgumentError: You must feed a value for placeholder tensor 'Placeholder_40' with dtype float and shape [?,784]
	 [[node Placeholder_40 (defined at <ipython-input-17-2d40799cd14b>:20)  = Placeholder[dtype=DT_FLOAT, shape=[?,784], _device="/job:localhost/replica:0/task:0/device:CPU:0"]()]]

Caused by op 'Placeholder_40', defined at:
  File "/usr/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/usr/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/frankzl/.envs/env36-ml/.venv/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/home/frankzl/.envs/env36-ml/.venv/lib/python3.6/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/home/frankzl/.envs/env36-ml/.venv/lib/python3.6/site-packages/ipykernel/kernelapp.py", line 505, in start
    self.io_loop.start()
  File "/home/frankzl/.envs/env36-ml/.